In [4]:
from qiskit import *
from qiskit.providers.aer import PulseSimulator, QasmSimulator
from qiskit.test.mock import FakeArmonk
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-research', group='Tharrmashastha-S', project='main')
backend_real = provider.get_backend('ibmq_armonk')
# backend_real = provider.get_backend('ibmq_casablanca')

from qiskit.providers.aer import PulseSimulator
from qiskit.providers.aer.pulse import PulseSystemModel
from qiskit.test.mock.backends.armonk.fake_armonk import FakeArmonk

# armonk_backend = FakeArmonk()

# freq_est = 4.97e9
# drive_est = 6.35e7
# armonk_backend.defaults().qubit_freq_est = [freq_est]
# armonk_backend.configuration().hamiltonian['h_str']= ['wq0*0.5*(I0-Z0)', 'omegad0*X0||D0']
# armonk_backend.configuration().hamiltonian['vars'] = {'wq0': 2 * np.pi * freq_est, 'omegad0': drive_est}
# armonk_backend.configuration().hamiltonian['qub'] = {'0': 2}
# armonk_backend.configuration().dt = 2.2222222222222221e-10

# armonk_model = PulseSystemModel.from_backend(armonk_backend)

# backend = PulseSimulator.from_backend(armonk_backend)

# backend = PulseSimulator.from_backend(backend_real)

# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds

# backend = QasmSimulator.from_backend(provider.get_backend('ibmq_armonk'))
# backend_real = provider.get_backend('ibmq_armonk')

backend_config = backend_real.configuration()
# print(backend_config.open_pulse)
dt = backend_config.dt
# dt = armonk_backend.configuration().dt
print('Sampling time : ' + str(dt/ns) + ' ns')

backend_defaults = backend_real.defaults()

import numpy as np

# We will find the qubit frequency for the following qubit.
qubit = 0

# The sweep will be centered around the estimated qubit frequency.
center_frequency_Hz = backend_defaults.qubit_freq_est[qubit]
# center_frequency_Hz = freq_est
# The default frequency is given in Hz
                                                                    # warning: this will change in a future release
# center_frequency_Hz = backend.qubit_freq_est[qubit]        # The default frequency is given in Hz
print(f"Qubit {qubit} has an estimated frequency of {center_frequency_Hz / GHz} GHz.")

# scale factor to remove factors of 10 from the data
scale_factor = 1e-14

# We will sweep 40 MHz around the estimated frequency
frequency_span_Hz = 40 * MHz
# in steps of 1 MHz.
frequency_step_Hz = 1 * MHz

# We will sweep 20 MHz above and 20 MHz below the estimated frequency
frequency_min = center_frequency_Hz - frequency_span_Hz / 2
frequency_max = center_frequency_Hz + frequency_span_Hz / 2
# Construct an np array of the frequencies for our experiment
frequencies_GHz = np.arange(frequency_min / GHz, 
                            frequency_max / GHz, 
                            frequency_step_Hz / GHz)

print(f"The sweep will go from {frequency_min / GHz} GHz to {frequency_max / GHz} GHz \
in steps of {frequency_step_Hz / MHz} MHz.")

from qiskit import pulse
from qiskit.pulse import Play, pulse_lib

# samples need to be multiples of 16
def get_closest_multiple_of_16(num):
    return int(num + 8 ) - (int(num + 8 ) % 16)

# Drive pulse parameters (us = microseconds)
drive_sigma_us = 0.075                     # This determines the actual width of the gaussian
drive_samples_us = drive_sigma_us*8        # This is a truncating parameter, because gaussians don't have 
                                            # a natural finite length

drive_sigma = get_closest_multiple_of_16(drive_sigma_us * us /dt)       # The width of the gaussian in units of dt
drive_samples = get_closest_multiple_of_16(drive_samples_us * us /dt)   # The truncating parameter in units of dt
drive_amp = 0.3
# Drive pulse samples
drive_pulse = pulse_lib.gaussian(duration=drive_samples,
                                 sigma=drive_sigma,
                                 amp=drive_amp,
                                 name='freq_sweep_excitation_pulse')

# Find out which group of qubits need to be acquired with this qubit
meas_map_idx = None
for i, measure_group in enumerate(backend_config.meas_map):
    if qubit in measure_group:
        meas_map_idx = i
        break
assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"

inst_shed_map = backend_real.defaults().instruction_schedule_map
measure = inst_shed_map.get('measure', qubits=backend_config.meas_map[meas_map_idx])


drive_chan = pulse.DriveChannel(qubit)
meas_chan = pulse.MeasureChannel(qubit)
acq_chan = pulse.AcquireChannel(qubit)


# Create the base schedule
# Start with drive pulse acting on the drive channel
schedule = pulse.Schedule(name='Frequency sweep')
schedule += Play(drive_pulse, drive_chan)
# The left shift `<<` is special syntax meaning to shift the start time of the schedule by some duration
schedule += measure << schedule.duration

# Create the frequency settings for the sweep (MUST BE IN HZ)
frequencies_Hz = frequencies_GHz*GHz
schedule_frequencies = [{drive_chan: freq} for freq in frequencies_Hz]

from qiskit import assemble

num_shots_per_frequency = 1024
frequency_sweep_program = assemble(schedule,
                                   backend=backend_real, 
                                   meas_level=1,
                                   meas_return='avg',
                                   shots=num_shots_per_frequency,
                                   schedule_los=schedule_frequencies)


job = backend_real.run(frequency_sweep_program)
# print(job.job_id())
from qiskit.tools.monitor import job_monitor
job_monitor(job)
frequency_sweep_results = job.result(timeout=120) # timeout parameter set to 120 seconds

import matplotlib.pyplot as plt

sweep_values = []
for i in range(len(frequency_sweep_results.results)):
    # Get the results from the ith experiment
    res = frequency_sweep_results.get_memory(i)*scale_factor
    # Get the results for `qubit` from this experiment
    sweep_values.append(res[qubit])

# plt.scatter(frequencies_GHz, np.real(sweep_values), color='black') # plot real part of sweep values
# plt.xlim([min(frequencies_GHz), max(frequencies_GHz)])
# plt.xlabel("Frequency [GHz]")
# plt.ylabel("Measured signal [a.u.]")
# plt.show()

from scipy.optimize import curve_fit

def fit_function(x_values, y_values, function, init_params):
    fitparams, conv = curve_fit(function, x_values, y_values, init_params)
    y_fit = function(x_values, *fitparams)
    
    return fitparams, y_fit

fit_params, y_fit = fit_function(frequencies_GHz,
                                 np.real(sweep_values), 
                                 lambda x, A, q_freq, B, C: (A / np.pi) * (B / ((x - q_freq)**2 + B**2)) + C,
                                 [-5, 4.975, 1, 3] # initial parameters for curve_fit
                                )

A, rough_qubit_frequency, B, C = fit_params
rough_qubit_frequency = rough_qubit_frequency*GHz # make sure qubit freq is in Hz
print(f"We've updated our qubit frequency estimate from "
      f"{round(backend_defaults.qubit_freq_est[qubit] / GHz, 5)} GHz to {round(rough_qubit_frequency/GHz, 5)} GHz.")


ibmqfactory.load_account:WARNING:2021-01-14 03:50:08,061: Credentials are already in use. The existing account in the session will be replaced.


Sampling time : 0.2222222222222222 ns
Qubit 0 has an estimated frequency of 4.974462023322946 GHz.
The sweep will go from 4.954462023322947 GHz to 4.994462023322947 GHz in steps of 1.0 MHz.


NameError: name 'sys_config' is not defined

In [64]:
# This experiment uses these values from the previous experiment:
    # `qubit`,
    # `measure`, and
    # `rough_qubit_frequency`.

# Rabi experiment parameters
num_rabi_points = 50

# Drive amplitude values to iterate over: 50 amplitudes evenly spaced from 0 to 0.75
drive_amp_min = 0
drive_amp_max = 0.75
drive_amps = np.linspace(drive_amp_min, drive_amp_max, num_rabi_points)


# Build the Rabi experiments:
#    A drive pulse at the qubit frequency, followed by a measurement,
#    where we vary the drive amplitude each time.
rabi_schedules = []
for drive_amp in drive_amps:
    rabi_pulse = pulse_lib.gaussian(duration=drive_samples, amp=drive_amp, 
                                    sigma=drive_sigma, name=f"Rabi drive amplitude = {drive_amp}")
    this_schedule = pulse.Schedule(name=f"Rabi drive amplitude = {drive_amp}")
    this_schedule += Play(rabi_pulse, drive_chan)
    # Reuse the measure instruction from the frequency sweep experiment
    this_schedule += measure << this_schedule.duration
    rabi_schedules.append(this_schedule)

# Assemble the schedules into a Qobj
num_shots_per_point = 1024

rabi_experiment_program = assemble(rabi_schedules,
                                   backend=backend,
                                   meas_level=1,
                                   meas_return='avg',
                                   shots=num_shots_per_point,
                                   schedule_los=[{drive_chan: rough_qubit_frequency}]
                                                * num_rabi_points)

print(job.job_id())
job = backend.run(rabi_experiment_program)
job_monitor(job)
rabi_results = job.result(timeout=120)

a8a40712-7be5-45e6-a755-659b4c092536
Job Status: job has successfully run


In [65]:
# center data around 0
def baseline_remove(values):
    return np.array(values) - np.mean(values)

rabi_values = []
for i in range(num_rabi_points):
    # Get the results for `qubit` from the ith experiment
    rabi_values.append(rabi_results.get_memory(i)[qubit]*scale_factor)

rabi_values = np.real(baseline_remove(rabi_values))

# plt.xlabel("Drive amp [a.u.]")
# plt.ylabel("Measured signal [a.u.]")
# plt.scatter(drive_amps, rabi_values, color='black') # plot real part of Rabi values
# plt.show()


fit_params, y_fit = fit_function(drive_amps,
                                 rabi_values, 
                                 lambda x, A, B, drive_period, phi: (A*np.cos(2*np.pi*x/drive_period - phi) + B),
                                 [3, 0.1, 0.5, 0])

# plt.scatter(drive_amps, rabi_values, color='black')
# plt.plot(drive_amps, y_fit, color='red')

drive_period = fit_params[2] # get period of rabi oscillation

# plt.axvline(drive_period/2, color='red', linestyle='--')
# plt.axvline(drive_period, color='red', linestyle='--')
# plt.annotate("", xy=(drive_period, 0), xytext=(drive_period/2,0), arrowprops=dict(arrowstyle="<->", color='red'))
# plt.annotate("$\pi$", xy=(drive_period/2-0.03, 0.1), color='red')

# plt.xlabel("Drive amp [a.u.]", fontsize=15)
# plt.ylabel("Measured signal [a.u.]", fontsize=15)
# plt.show()

pi_amp = abs(drive_period / 2)
print(f"Pi Amplitude = {pi_amp}")

pi_pulse = pulse_lib.gaussian(duration=drive_samples,
                              amp=pi_amp, 
                              sigma=drive_sigma,
                              name='pi_pulse')

# Create two schedules

# Ground state schedule
gnd_schedule = pulse.Schedule(name="ground state")
gnd_schedule += measure

# Excited state schedule
exc_schedule = pulse.Schedule(name="excited state")
exc_schedule += Play(pi_pulse, drive_chan)  # We found this in Part 2A above
exc_schedule += measure << exc_schedule.duration

# Execution settings
num_shots = 1024

gnd_exc_program = assemble([gnd_schedule, exc_schedule],
                           backend=backend,
                           meas_level=1,
                           meas_return='single',
                           shots=num_shots,
                           schedule_los=[{drive_chan: rough_qubit_frequency}] * 2)

# print(job.job_id())

# from qiskit.providers.aer.noise import NoiseModel
# noise_model = NoiseModel.from_backend(backend_real)


job = backend.run(gnd_exc_program)
job_monitor(job)

gnd_exc_results = job.result(timeout=120)

Pi Amplitude = 0.2644116472309442
Job Status: job is being initialized

/home/tharrma/research_qiskit/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Job Status: job has successfully run


In [77]:
from sklearn import svm
from sklearn.model_selection import train_test_split

def classify(gnd_exc_results):
    gnd_results = gnd_exc_results.get_memory(0)[:, qubit]*scale_factor
    exc_results = gnd_exc_results.get_memory(1)[:, qubit]*scale_factor
    X = []
    y = []
    for res in gnd_results:
        X.append([np.real(res), np.imag(res)])
        y.append(0)

    for res in exc_results:
        X.append([np.real(res), np.imag(res)])
        y.append(1)

    X = np.asarray(X)
    y = np.asarray(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    clf = svm.SVC()
    clf.fit(X_train, y_train)
    return (np.asarray(clf.predict(X_test))==y_test).all()

In [78]:
classify(gnd_exc_results)

True